<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/LGBM_Model__Stock_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector


In [9]:
# Loading data
file_path = Path("HPQ.csv")
df = pd.read_csv(file_path)
df.dropna(inplace=True)
df=df[df.FutureTrend !='Neutral']
df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,29.180000,29.510000,28.959999,29.200001,7914000,0.436366,0.068542,Negative,Positive,Positive,0,Negative
750,24/06/2021,29.500000,29.510000,29.110001,29.469999,6270700,0.899998,-0.101697,Negative,Negative,Negative,0,Negative
751,25/06/2021,29.400000,29.920000,29.370001,29.709999,17050100,0.618179,1.054420,Positive,Negative,Positive,0,Negative
752,28/06/2021,29.760000,30.030001,29.639999,29.809999,6570000,0.435896,0.168008,Negative,Positive,Positive,0,Negative
753,29/06/2021,29.950001,30.320000,29.790001,30.139999,7779100,0.660376,0.634386,Negative,Negative,Positive,0,Negative


In [10]:
#OHLC Chart with Volume
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Ohlc(x=df.Date,
                      open=df.Open,
                      high=df.High,
                      low=df.Low,
                      close=df.Close,
                      name='Price'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.Date, y=df.Volume, name='Volume'), row=2, col=1)

fig.update_layout(title_text="HPQ Volume")
fig.update(layout_xaxis_rangeslider_visible=False) 
fig.show()

In [11]:
target=df['FutureTrend']
df=df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
df= df.replace(['Negative','Positive',  'Neutral'],[0,1,2])
appl_df =df.set_index('Date')
from imblearn.over_sampling import SMOTE
# pd.DataFrame(oe_df)
oversample = SMOTE()
appl_df, target = oversample.fit_resample(appl_df, target)


In [12]:
%pip install lgbm
import lightgbm as lgbm
new_df=appl_df
new_df

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
  # ('one-hot-encoder', categorical_preprocessor, categorical_columns),
   ('standard-scaler', numerical_preprocessor, numerical_columns)
    ])

model_lgbm = make_pipeline(preprocessor, lgbm.LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=70,
                                learning_rate=0.4, n_estimators=150, max_depth=4,
                                bagging_fraction=0.8, feature_fraction=0.9,reg_alpha=0.7
                                ,reg_lambda=0.6))
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2,random_state=42)
model_lgbm.fit(X_train,y_train)
print("Accuracy score (training): {0:.3f}".format(
        model_lgbm.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_lgbm.score(
            X_test, 
            y_test)))

                        

ERROR: Could not find a version that satisfies the requirement lgbm (from versions: none)
ERROR: No matching distribution found for lgbm
[]
Accuracy score (training): 0.997
Accuracy score (validation): 0.545


In [13]:
import lightgbm
%pip install bayesian-optimization
def lgb_classifier( num_leaves, max_depth, 
                   lambda_l2, lambda_l1, min_child_samples, 
                   min_data_in_leaf):
    params = {"boosting_type": 'gbdt',
              "objective" : "binary",
              "metric" : "auc", 
              "is_unbalance": True,
              "num_leaves" : int(num_leaves),
              "max_depth" : int(max_depth),
              "lambda_l2" : lambda_l2,
              "lambda_l1" : lambda_l1,
              "num_threads" : 20,
              "min_child_samples" : int(min_child_samples),
              "min_data_in_leaf": int(min_data_in_leaf),
              "learning_rate" : 0.01,
              "subsample_freq" : 5,
              "bagging_seed" : 42,
              "verbosity" : -1,
             }
    train_data = lightgbm.Dataset(X_train, y_train)
    cv_result = lightgbm.cv(params,
                       train_data,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Hyper Parameter tuning 

In [14]:
from bayes_opt import BayesianOptimization
lgbBO = BayesianOptimization(lgb_classifier, {  
                                                'num_leaves': (25, 4000),
                                                'max_depth': (5, 40),
                                                'lambda_l2': (0.0, 0.1),
                                                'lambda_l1': (0.0, 0.1),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=70, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.5      |  0.09423  |  0.08929  |  28.76    |  2.033e+0 |  1.118e+0 |  2.946e+0 |
|  2        |  0.5      |  0.05682  |  0.07753  |  25.93    |  9.971e+0 |  1.009e+0 |  1.272e+0 |
|  3        |  0.5337   |  0.03707  |  0.05641  |  38.03    |  324.8    |  165.6    |  106.3    |
|  4        |  0.5      |  0.03531  |  0.0896   |  26.11    |  5.928e+0 |  909.8    |  3.67e+03 |
|  5        |  0.5      |  0.09101  |  0.06862  |  15.54    |  9.742e+0 |  852.0    |  3.574e+0 |
|  6        |  0.5      |  0.007494 |  0.09631  |  20.44    |  401.5    |  328.5    |  34.22    |
|  7        |  0.5023   |  0.03274  |  0.009453 |  16.3     |  5.248e+0 |  220.5    |  2.658e+0 |
|  8        |  0.5      |  0.04222  |  0.007757 |  10.88    |  5.062e+0 |  482.7    |  1.107e+0 |
|  9        |  0.5  

In [15]:

model_2 = make_pipeline(preprocessor,  lgbm.LGBMClassifier(n_estimators=38,
                                      learning_rate=0.3,
                                      max_features=6,
                                      max_depth=5,
                                      random_state=4,subsample=1,
                                      criterion='friedman_mse'))
                        
print(new_df.shape,target.shape)
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)
model_2.fit(X_train,y_train)

  # Scoring the model_2
print("Accuracy score (training): {0:.3f}".format(
        model_2.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_2.score(
            X_test, 
            y_test)))

(824, 11) (824,)
Accuracy score (training): 0.909
Accuracy score (validation): 0.564
